# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json
# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,burutu,5.3509,5.5076,28.16,73,54,5.28,NG,1696177455
1,1,hermanus,-34.4187,19.2345,17.31,65,0,6.15,ZA,1696177455
2,2,bikenibeu village,1.3673,173.1241,27.98,83,40,0.51,KI,1696177455
3,3,island harbour,50.7240,-1.2758,20.52,89,99,2.24,GB,1696177343
4,4,turkmenbasy,40.0222,52.9552,21.43,24,0,2.06,TM,1696177455


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
humidity_map_plot = city_data_df.hvplot.points(
'Lng',
'Lat',
 geo = True,
 size = 'Humidity',
 tiles = 'OSM'
)

# Display the map
humidity_map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_city_data_df = city_data_df.loc[(city_data_df['Max Temp']<27) & (city_data_df['Max Temp']>16) & city_data_df['Humidity']<75]
# Drop any rows with null values
ideal_city_data_df = ideal_city_data_df.dropna()
# Display sample data
ideal_city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,burutu,5.3509,5.5076,28.16,73,54,5.28,NG,1696177455
1,1,hermanus,-34.4187,19.2345,17.31,65,0,6.15,ZA,1696177455
2,2,bikenibeu village,1.3673,173.1241,27.98,83,40,0.51,KI,1696177455
3,3,island harbour,50.7240,-1.2758,20.52,89,99,2.24,GB,1696177343
4,4,turkmenbasy,40.0222,52.9552,21.43,24,0,2.06,TM,1696177455
...,...,...,...,...,...,...,...,...,...,...
555,555,praia da vitoria,38.7333,-27.0667,22.64,68,40,4.12,PT,1696177619
556,556,olyokminsk,60.3736,120.4264,5.22,90,53,1.10,RU,1696177619
557,557,magdagachi,53.4500,125.8000,4.87,80,85,1.28,RU,1696177619
558,558,santo angelo,-28.2992,-54.2631,21.39,50,0,3.24,BR,1696177495


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_city_data_df[['City', 'Country','Lat','Lng','Humidity']].copy()
hotel_df
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,burutu,NG,5.3509,5.5076,73,
1,hermanus,ZA,-34.4187,19.2345,65,
2,bikenibeu village,KI,1.3673,173.1241,83,
3,island harbour,GB,50.7240,-1.2758,89,
4,turkmenbasy,TM,40.0222,52.9552,24,
...,...,...,...,...,...,...
555,praia da vitoria,PT,38.7333,-27.0667,68,
556,olyokminsk,RU,60.3736,120.4264,90,
557,magdagachi,RU,53.4500,125.8000,80,
558,santo angelo,BR,-28.2992,-54.2631,50,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [20]:
# Set parameters to search for a hotel
radius = 10000
#filters = f"circle:{longitude},{latitude},{radius}"
#bias = f"proximity:{longitude},{latitude}"
categories = "accommodation.hotel"
params = {
    "categories":categories,
    "apiKey":geoapify_key
}


# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index,"Lat"]
    longitude = hotel_df.loc[index,"Lng"]
    city = hotel_df.loc[index,"City"]
   
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    

   
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    response = requests.get(base_url, params=params)
    

    # Convert the API response to JSON format
    name_address = response.json()
    #print(json.dumps(name_address, indent=4, sort_keys=True))
    

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df


Starting hotel search
burutu - nearest hotel: No hotel found
hermanus - nearest hotel: Aloe guest house
bikenibeu village - nearest hotel: Otintaai Hotel
island harbour - nearest hotel: Premier Inn
turkmenbasy - nearest hotel: Türkmenbaşy Oteli
yei - nearest hotel: New Tokyo
whitehorse - nearest hotel: Town & Mountain Hotel
adamstown - nearest hotel: No hotel found
sisimiut - nearest hotel: Sømandshjemmet
grytviken - nearest hotel: No hotel found
margaret river - nearest hotel: Margaret River Hotel
bulungu - nearest hotel: No hotel found
avarua - nearest hotel: Paradise Inn
georgetown - nearest hotel: Page 63 hostel
fale old settlement - nearest hotel: No hotel found
knyaze-volkonskoye - nearest hotel: No hotel found
new norfolk - nearest hotel: Woodbridge on the Derwent
blackmans bay - nearest hotel: Villa Howden
sibolga - nearest hotel: Hotel Prima Indah
margate - nearest hotel: The Reading Rooms
ilulissat - nearest hotel: Best Western Plus Hotel Ilulissat
albany - nearest hotel: No 

leava - nearest hotel: Fia fia
comayagua - nearest hotel: Hotel Comayagua Colonial
kingston - nearest hotel: Aphrodite Hotel
aasiaat - nearest hotel: SØMA
st. john's - nearest hotel: Alt Hotel
tallkayf - nearest hotel: No hotel found
linfen - nearest hotel: 7天连锁酒店
cangzhou - nearest hotel: No hotel found
geidam - nearest hotel: No hotel found
puerto ayora - nearest hotel: Hostal La Mirada De Solitario George
constantia - nearest hotel: Hotel Chérica
teknaf - nearest hotel: No hotel found
kulhudhuffushi - nearest hotel: No hotel found
mar del plata - nearest hotel: Nuevo Ostende
arraial do cabo - nearest hotel: No hotel found
north platte - nearest hotel: Howard Johnson Inn - North Platte
veraval - nearest hotel: Shyam
surigao - nearest hotel: S.E.C. Mini Hotel & Pool
hamilton - nearest hotel: North Vista Manor
galvez - nearest hotel: El Portal
vorgashor - nearest hotel: София
santa ana - nearest hotel: Holiday Inn Express
afaahiti - nearest hotel: Omati Lodge
cururupu - nearest hotel: 

harstad - nearest hotel: Scandic Harstad
lyubinskiy - nearest hotel: No hotel found
joao pinheiro - nearest hotel: Lícia
vila franca do campo - nearest hotel: Vinha d'Areia Beach Hotel
wuyishan - nearest hotel: No hotel found
portland - nearest hotel: the Hoxton
puerto baquerizo moreno - nearest hotel: Dory's House
sitangkai - nearest hotel: No hotel found
fisterra - nearest hotel: Hotel Temático do Banco Azul
port antonio - nearest hotel: No hotel found
pekanbaru - nearest hotel: Hotel Radja
floro - nearest hotel: Comfort Hotel Victoria Florø
rudnichnyy - nearest hotel: Общежитие
meridian - nearest hotel: Best Western
port lincoln - nearest hotel: Boston Hotel
hammerfest - nearest hotel: Thon Hotel Hammerfest
nsanje - nearest hotel: No hotel found
marsh harbour - nearest hotel: Lofty Fig Villas
cabedelo - nearest hotel: No hotel found
tutoia - nearest hotel: Pousada San Vicente
susuman - nearest hotel: No hotel found
vernon - nearest hotel: Okanagan Royal Park Inn
emkhomazi - nearest 

,City,Country,Lat,Lng,Humidity,Hotel Name
0,burutu,NG,5.3509,5.5076,73,No hotel found
1,hermanus,ZA,-34.4187,19.2345,65,Aloe guest house
2,bikenibeu village,KI,1.3673,173.1241,83,Otintaai Hotel
3,island harbour,GB,50.7240,-1.2758,89,Premier Inn
4,turkmenbasy,TM,40.0222,52.9552,24,Türkmenbaşy Oteli
...,...,...,...,...,...,...
555,praia da vitoria,PT,38.7333,-27.0667,68,Salles
556,olyokminsk,RU,60.3736,120.4264,90,Колос
557,magdagachi,RU,53.4500,125.8000,80,Восток-Сервис
558,santo angelo,BR,-28.2992,-54.2631,50,Maerkli Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [22]:
%%capture --no-display

# Configure the map plot
hotels_map = hotel_df.hvplot.points(
'Lng',
'Lat',
 geo = True,
 tiles = 'OSM',
 hover_cols = ["Hotel Name","Country"]
 
)
# Display the map
hotels_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Hotel Name,Country)